In [5]:
# ===============================
# Connect Four Simulation Notebook
# ===============================

import sys
import os
import numpy as np
from itertools import combinations

# --- Make sure SRC folder is on path ---
sys.path.append(os.path.join(os.getcwd(), "src"))

# --- Import from SRC ---
from board import create_board, make_move, check_win
from player import random_player, heuristic_player, intelligent_player
from simulation import simulate_game

BASE_SIM = 100

players = {
    "4 Years": random_player,
    "7 Years": heuristic_player,
    "9 Years": lambda b, p: intelligent_player(b, p, depth=1),
    "11 Years": lambda b, p: intelligent_player(b, p, depth=2)
}

def determine_sim_count(balance):
    return int(np.clip(BASE_SIM * (1 + balance * 2), 100, 500))

balance_estimates = {
    ("4 Years","7 Years"): 0.1,
    ("4 Years","9 Years"): 0.3,
    ("4 Years","11 Years"): 0.05,
    ("7 Years","9 Years"): 0.5,
    ("7 Years","11 Years"): 0.7,
    ("9 Years","11 Years"): 0.8
}

for p1, p2 in combinations(players.keys(), 2):

    sim_count = determine_sim_count(balance_estimates.get((p1, p2), 0.3))
    outcomes = {p1: 0, p2: 0, "Draw": 0}

    for _ in range(sim_count):
        winner = simulate_game(players[p1], players[p2])
        if winner == 1:
            outcomes[p1] += 1
        elif winner == 2:
            outcomes[p2] += 1
        else:
            outcomes["Draw"] += 1

    p1_prob = outcomes[p1] / sim_count
    p2_prob = outcomes[p2] / sim_count
    d_prob = outcomes["Draw"] / sim_count

    se1 = np.sqrt(p1_prob * (1 - p1_prob) / sim_count)
    se2 = np.sqrt(p2_prob * (1 - p2_prob) / sim_count)
    sed = np.sqrt(d_prob * (1 - d_prob) / sim_count)

    q1 = 1 / p1_prob if p1_prob > 0 else float("inf")
    q2 = 1 / p2_prob if p2_prob > 0 else float("inf")
    qd = 1 / d_prob if d_prob > 0 else float("inf")

    print(f"\n{p1} vs {p2} ({sim_count} games)")
    print(f"{p1}: {p1_prob*100:.1f}% ± {se1*100:.1f}% → Quote ≈ {q1:.2f}")
    print(f"{p2}: {p2_prob*100:.1f}% ± {se2*100:.1f}% → Quote ≈ {q2:.2f}")
    print(f"Draw: {d_prob*100:.1f}% ± {sed*100:.1f}% → Quote ≈ {qd:.2f}")



ModuleNotFoundError: No module named 'src'